In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az
import re

In [2]:
full = pd.read_csv("Data/full_data.csv")
full.head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,All_NBA_Pos,All_NBA_Team,All_NBA_Pts_Won,All_NBA_Pts_Max,All_NBA_Share,All_NBA_1st_Team_Votes,All_NBA_2nd_Team_Votes,All_NBA_3rd_Team_Votes,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False


In [3]:
## Remove All ALL Stats except boolean All_Stars
remove_all_star = full.loc[:, ~full.columns.str.startswith('All_')]
remove_all_star["All_NBA_Boolean"] = full["All_NBA_Boolean"]
remove_all_star.head()

/var/folders/pn/dgy7ckd90nl7mlj6g6rc_1kw0000gn/T/ipykernel_3945/1871535054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_all_star["All_NBA_Boolean"] = full["All_NBA_Boolean"]


,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player_id,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,2.0,2.9,0.082,-1.9,-0.2,-2.1,0.0,abdulka01,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,0.5,0.9,0.066,-3.0,-0.4,-3.4,-0.2,acresma01,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,2.9,8.3,0.143,2.7,0.2,2.9,3.4,adamsmi01,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,1.0,2.2,0.093,-1.1,-0.8,-2.0,0.0,alarima01,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,0.0,0.0,-0.035,-5.5,-1.7,-7.2,-0.1,allenra01,1988-1989,False


In [4]:
remove_nan = remove_all_star.dropna()
remove_nan

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player_id,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,2.0,2.9,0.082,-1.9,-0.2,-2.1,0.0,abdulka01,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,0.5,0.9,0.066,-3.0,-0.4,-3.4,-0.2,acresma01,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,2.9,8.3,0.143,2.7,0.2,2.9,3.4,adamsmi01,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,1.0,2.2,0.093,-1.1,-0.8,-2.0,0.0,alarima01,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,0.0,0.0,-0.035,-5.5,-1.7,-7.2,-0.1,allenra01,1988-1989,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15438,Tremont Waters,PG,24,TOT,3,50,5.4,0.357,0.643,0.000,...,0.1,0.0,-0.042,-7.8,0.1,-7.7,-0.1,watertr01,2021-2022,False
15439,Derrick White,SG,27,TOT,75,2199,14.8,0.549,0.461,0.294,...,2.3,5.1,0.111,0.4,0.6,0.9,1.6,whitede01,2021-2022,False
15440,Justise Winslow,SF,25,TOT,48,774,13.1,0.479,0.257,0.257,...,1.1,0.8,0.048,-2.7,1.0,-1.7,0.1,winslju01,2021-2022,False
15441,Moses Wright,PF,23,TOT,4,14,18.3,0.470,0.250,0.750,...,0.0,0.1,0.180,1.1,0.4,1.4,0.0,wrighmo01,2021-2022,False


In [5]:
pd.factorize(remove_nan.year)

(array([ 0,  0,  0, ..., 33, 33, 33]),
 Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
        '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
        '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
        '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
        '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
        '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
       dtype='object'))

In [6]:
season_idxs, seasons = pd.factorize(remove_nan.year)
coords = {
    "season": seasons,
    "obs_id": np.arange(len(season_idxs)),
}
coords

{'season': Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
        '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
        '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
        '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
        '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
        '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
       dtype='object'),
 'obs_id': array([    0,     1,     2, ..., 15379, 15380, 15381])}

In [7]:
season_idxs

array([ 0,  0,  0, ..., 33, 33, 33])

In [8]:
seasons

Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
       '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
       '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
       '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
       '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
       '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
       '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
      dtype='object')

In [9]:
import theano.tensor as T
def logistic(l):
    return 1 / (1 + T.exp(-l))

In [10]:
remove_nan.columns[1:-2]

Index(['Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'player_id'],
      dtype='object')

In [11]:
numeric_col = pd.DataFrame(remove_nan.dtypes[:-1]).reset_index()
numeric_col = numeric_col[numeric_col[0] != 'object']
numeric_col.columns = ['name', 'type']
numeric_col

,name,type
2,Age,int64
4,G,int64
5,MP,int64
6,PER,float64
7,TS%,float64
8,3PAr,float64
9,FTr,float64
10,ORB%,float64
11,DRB%,float64
12,TRB%,float64


In [12]:
for i,k in enumerate(numeric_col.name[0:5]):
    print(i,k)

0 Age
1 G
2 MP
3 PER
4 TS%


In [13]:
master_df = []
for i,col in enumerate(numeric_col.name):
    with pm.Model(coords=coords) as hierarchical_model:
        season_idx = pm.Data("season_idx", season_idxs, dims="obs_id")
        # Hyperpriors for group nodes
        mu_a = pm.Normal("mu_a", mu=0.0, sigma=10000)
        sigma_a = pm.HalfNormal("sigma_a", 5.0)
        mu_b = pm.Normal("mu_b", mu=0.0, sigma=10000)
        sigma_b = pm.HalfNormal("sigma_b", 5.0)


        # Intercept for each county, distributed around group mean mu_a
        # Above we just set mu and sd to a fixed value while here we
        # plug in a common group distribution for all a and b (which are
        # vectors of length n_counties).
        a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="season")
        # effect difference between basement and floor level
        b = pm.Normal("b", mu=mu_b, sigma=sigma_b, dims="season")

        nba_est = a[season_idx] + b[season_idx] * remove_nan[col].values

        # Data likelihood
        nba_like = pm.Bernoulli(
            "nba_like",
            # p=logistic(nba_est),
            logit_p = nba_est,
            observed=remove_nan.All_NBA_Boolean.astype(int).values
        )
    # with hierarchical_model:
        hierarchical_trace = pm.sample(100, tune=100, target_accept=0.6, return_inferencedata=True)
        df = pm.summary(hierarchical_trace).reset_index().rename(columns={"index":"distribution"})
        df['predictor'] = [col]*len(df)
        master_df.append(df)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a, sigma_b, mu_b, sigma_a, mu_a]


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 28 seconds.
There were 65 divergences after tuning. Increase `target_accept` or reparameterize.
There were 19 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8181222278726574, but should be close to 0.6. Try to increase the number of tuning steps.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.2431815682382698, but should be close to 0.6. Try to increase the number of tuning steps.
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8086699390993161, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 23 seconds.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.12936232852765736, but should be close to 0.6. Try to increase the number of tuning steps.
There were 97 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.35061075139723225, but should be close to 0.6. Try to increase the number of tuning steps.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 5.084629743887716e-05, but should be close to 0.6. Try to increase the number of tuning steps.
There were 39 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 12 seconds.
There were 47 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 2.872844695430157e-32, but should be close to 0.6. Try to increase the number of tuning steps.
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.03, but should be close to 0.6. Try to increase the number of tuning steps.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.3334389992475718, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 20 seconds.
The acceptance probability does not match the target. It is 0.8593662859743313, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7616946379480575, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7617579202943588, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.779669122632633, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The number of effective samples is smaller than 25% for some parameters.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess 

...

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 30 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8469841953592949, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.984228167441322, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7592125642931584, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8076069818580287, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.2 for some parameters.
The number of effective samples is smaller than 10% for some parameters.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using 

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 17 seconds.
There were 54 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.46576865270461204, but should be close to 0.6. Try to increase the number of tuning steps.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8440593519461718, but should be close to 0.6. Try to increase the number of tuning steps.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7143223016439607, but should be close to 0.6. Try to increase the number of tuning steps.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The 

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 19 seconds.
There were 51 divergences after tuning. Increase `target_accept` or reparameterize.
There were 27 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8296867123262266, but should be close to 0.6. Try to increase the number of tuning steps.
There were 53 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8584120384710352, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 14 seconds.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.720840904174851, but should be close to 0.6. Try to increase the number of tuning steps.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7380606376593094, but should be close to 0.6. Try to increase the number of tuning steps.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7378615261802338, but should be close to 0.6. Try to increase the number of tuning steps.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The num

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 24 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9283646986245968, but should be close to 0.6. Try to increase the number of tuning steps.
There were 37 divergences after tuning. Increase `target_accept` or reparameterize.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8572349899763912, but should be close to 0.6. Try to increase the number of tuning steps.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7709172211411667, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The n

In [14]:
final_master_df = pd.concat(master_df)
final_master_df.head(10)

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor
0,mu_a,-1.230,0.300,-1.612,-0.867,0.140,0.106,5.0,12.0,2.69,Age
1,mu_b,-0.079,0.011,-0.095,-0.064,0.005,0.004,5.0,46.0,2.35,Age
2,a[1988-1989],-1.234,0.278,-1.576,-0.868,0.126,0.097,5.0,31.0,2.61,Age
3,a[1989-1990],-1.214,0.316,-1.610,-0.748,0.145,0.111,5.0,25.0,2.42,Age
4,a[1990-1991],-1.259,0.338,-1.715,-0.853,0.157,0.120,5.0,30.0,2.54,Age
5,a[1991-1992],-1.234,0.339,-1.683,-0.796,0.156,0.120,5.0,17.0,2.54,Age
6,a[1992-1993],-1.217,0.304,-1.598,-0.816,0.140,0.106,5.0,14.0,2.37,Age
7,a[1993-1994],-1.197,0.334,-1.599,-0.742,0.155,0.118,5.0,19.0,2.56,Age
8,a[1994-1995],-1.251,0.333,-1.714,-0.859,0.155,0.119,5.0,13.0,2.51,Age
9,a[1995-1996],-1.207,0.289,-1.549,-0.802,0.132,0.100,5.0,18.0,2.48,Age


In [15]:
final_master_df['significant'] = ((final_master_df["hdi_3%"] < 0) & (final_master_df["hdi_97%"] > 0)) == False
final_master_df['typeDist'] = np.where(final_master_df['distribution'].str.contains('b'), 'beta', 'intercept')
final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
0,mu_a,-1.230,0.300,-1.612,-0.867,0.140,0.106,5.0,12.0,2.69,Age,True,intercept
1,mu_b,-0.079,0.011,-0.095,-0.064,0.005,0.004,5.0,46.0,2.35,Age,True,beta
2,a[1988-1989],-1.234,0.278,-1.576,-0.868,0.126,0.097,5.0,31.0,2.61,Age,True,intercept
3,a[1989-1990],-1.214,0.316,-1.610,-0.748,0.145,0.111,5.0,25.0,2.42,Age,True,intercept
4,a[1990-1991],-1.259,0.338,-1.715,-0.853,0.157,0.120,5.0,30.0,2.54,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,b[2019-2020],1.871,0.140,1.621,2.112,0.011,0.008,157.0,344.0,1.05,VORP,True,beta
68,b[2020-2021],1.834,0.146,1.601,2.138,0.030,0.022,27.0,87.0,1.11,VORP,True,beta
69,b[2021-2022],1.799,0.135,1.578,2.049,0.025,0.018,32.0,105.0,1.11,VORP,True,beta
70,sigma_a,0.378,0.113,0.207,0.577,0.023,0.016,22.0,112.0,1.14,VORP,True,intercept


In [16]:
season_final_master_df = final_master_df[final_master_df["distribution"].str.contains('\[')]
season_final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
2,a[1988-1989],-1.234,0.278,-1.576,-0.868,0.126,0.097,5.0,31.0,2.61,Age,True,intercept
3,a[1989-1990],-1.214,0.316,-1.610,-0.748,0.145,0.111,5.0,25.0,2.42,Age,True,intercept
4,a[1990-1991],-1.259,0.338,-1.715,-0.853,0.157,0.120,5.0,30.0,2.54,Age,True,intercept
5,a[1991-1992],-1.234,0.339,-1.683,-0.796,0.156,0.120,5.0,17.0,2.54,Age,True,intercept
6,a[1992-1993],-1.217,0.304,-1.598,-0.816,0.140,0.106,5.0,14.0,2.37,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,b[2017-2018],1.665,0.126,1.447,1.882,0.019,0.013,42.0,154.0,1.07,VORP,True,beta
66,b[2018-2019],1.649,0.124,1.375,1.851,0.026,0.019,23.0,34.0,1.12,VORP,True,beta
67,b[2019-2020],1.871,0.140,1.621,2.112,0.011,0.008,157.0,344.0,1.05,VORP,True,beta
68,b[2020-2021],1.834,0.146,1.601,2.138,0.030,0.022,27.0,87.0,1.11,VORP,True,beta


In [17]:
final_master_df.groupby(['predictor', 'typeDist']).sum()['significant']

predictor  typeDist 
3PAr       beta         36
           intercept    36
AST%       beta         36
           intercept    36
Age        beta         36
           intercept    36
BLK%       beta         36
           intercept    36
BPM        beta         35
           intercept    36
DBPM       beta         35
           intercept    36
DRB%       beta         36
           intercept    36
DWS        beta         36
           intercept    36
FTr        beta         36
           intercept    36
G          beta         36
           intercept    36
MP         beta         32
           intercept     4
OBPM       beta         35
           intercept    36
ORB%       beta          1
           intercept    36
OWS        beta         36
           intercept    36
PER        beta         36
           intercept    36
STL%       beta         36
           intercept    36
TOV%       beta         36
           intercept    36
TRB%       beta         36
           intercept    36
TS%    

In [18]:
season_final_master_df.groupby(['predictor', 'typeDist']).sum()['significant']

predictor  typeDist 
3PAr       beta         34
           intercept    34
AST%       beta         34
           intercept    34
Age        beta         34
           intercept    34
BLK%       beta         34
           intercept    34
BPM        beta         33
           intercept    34
DBPM       beta         33
           intercept    34
DRB%       beta         34
           intercept    34
DWS        beta         34
           intercept    34
FTr        beta         34
           intercept    34
G          beta         34
           intercept    34
MP         beta         31
           intercept     3
OBPM       beta         33
           intercept    34
ORB%       beta          0
           intercept    34
OWS        beta         34
           intercept    34
PER        beta         34
           intercept    34
STL%       beta         34
           intercept    34
TOV%       beta         34
           intercept    34
TRB%       beta         34
           intercept    34
TS%    

In [19]:
season_final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
2,a[1988-1989],-1.234,0.278,-1.576,-0.868,0.126,0.097,5.0,31.0,2.61,Age,True,intercept
3,a[1989-1990],-1.214,0.316,-1.610,-0.748,0.145,0.111,5.0,25.0,2.42,Age,True,intercept
4,a[1990-1991],-1.259,0.338,-1.715,-0.853,0.157,0.120,5.0,30.0,2.54,Age,True,intercept
5,a[1991-1992],-1.234,0.339,-1.683,-0.796,0.156,0.120,5.0,17.0,2.54,Age,True,intercept
6,a[1992-1993],-1.217,0.304,-1.598,-0.816,0.140,0.106,5.0,14.0,2.37,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,b[2017-2018],1.665,0.126,1.447,1.882,0.019,0.013,42.0,154.0,1.07,VORP,True,beta
66,b[2018-2019],1.649,0.124,1.375,1.851,0.026,0.019,23.0,34.0,1.12,VORP,True,beta
67,b[2019-2020],1.871,0.140,1.621,2.112,0.011,0.008,157.0,344.0,1.05,VORP,True,beta
68,b[2020-2021],1.834,0.146,1.601,2.138,0.030,0.022,27.0,87.0,1.11,VORP,True,beta


In [20]:
len(np.unique(remove_nan.year))

34

In [21]:
with pm.Model(coords=coords) as hierarchical_model:
    season_idx = pm.Data("season_idx", season_idxs, dims="obs_id")
    # Hyperpriors for group nodes
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=10000)
    sigma_a = pm.HalfNormal("sigma_a", 5.0)

    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="season")

    nba_est = a[season_idx]
    for i,col in enumerate(numeric_col.name):
        mu = pm.Normal("mu_b{}".format(i), mu=0.0, sigma=10000)
        sigma = pm.HalfNormal("sigma_b{}".format(i), 5.0)
        var = pm.Normal("b{}".format(i), mu=mu, sigma=sigma, dims="season")
        nba_est = nba_est + var[season_idx] * remove_nan[col].values

    # Data likelihood
    nba_like = pm.Bernoulli(
        "nba_like",
        # p=logistic(nba_est),
        logit_p = nba_est,
        observed=remove_nan.All_NBA_Boolean.astype(int).values
    )

In [22]:
with hierarchical_model:
    hierarchical_trace = pm.sample(100, tune=100, target_accept=0.6, return_inferencedata=True)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b22, sigma_b22, mu_b22, b21, sigma_b21, mu_b21, b20, sigma_b20, mu_b20, b19, sigma_b19, mu_b19, b18, sigma_b18, mu_b18, b17, sigma_b17, mu_b17, b16, sigma_b16, mu_b16, b15, sigma_b15, mu_b15, b14, sigma_b14, mu_b14, b13, sigma_b13, mu_b13, b12, sigma_b12, mu_b12, b11, sigma_b11, mu_b11, b10, sigma_b10, mu_b10, b9, sigma_b9, mu_b9, b8, sigma_b8, mu_b8, b7, sigma_b7, mu_b7, b6, sigma_b6, mu_b6, b5, sigma_b5, mu_b5, b4, sigma_b4, mu_b4, b3, sigma_b3, mu_b3, b2, sigma_b2, mu_b2, b1, sigma_b1, mu_b1, b0, sigma_b0, mu_b0, a, sigma_a, mu_a]


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 266 seconds.
There were 65 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 84 divergences after tuning. Increase `target_accept` or reparameterize.
There were 92 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.40596284342930367, but should be close to 0.6. Try to increase the number of tuning steps.
There were 73 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.


In [23]:
hierarchical_trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data

In [24]:
all_predictor_df = pm.summary(hierarchical_trace).reset_index().rename(columns={"index":"distribution"})
all_predictor_df['significant'] = ((all_predictor_df["hdi_3%"] < 0) & (all_predictor_df["hdi_97%"] > 0)) == False
all_predictor_df['typeDist'] = np.where(all_predictor_df['distribution'].str.contains('b'), 'beta', 'intercept')
all_predictor_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist
0,mu_a,0.404,0.459,-0.441,0.862,0.220,0.167,5.0,15.0,3.13,False,intercept
1,a[1988-1989],0.612,0.327,0.075,0.931,0.156,0.119,5.0,15.0,3.42,True,intercept
2,a[1989-1990],-0.078,0.537,-0.582,0.824,0.257,0.196,5.0,12.0,3.38,False,intercept
3,a[1990-1991],-0.256,0.584,-0.687,0.762,0.280,0.213,5.0,13.0,2.28,False,intercept
4,a[1991-1992],0.681,0.184,0.382,0.870,0.086,0.065,5.0,14.0,3.07,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...
859,sigma_b18,3.802,2.078,1.509,8.009,0.971,0.736,5.0,15.0,3.14,True,beta
860,sigma_b19,3.814,1.535,2.172,5.971,0.726,0.554,5.0,15.0,2.62,True,beta
861,sigma_b20,3.577,1.680,1.711,5.789,0.800,0.608,5.0,14.0,2.96,True,beta
862,sigma_b21,4.047,1.738,2.498,8.378,0.808,0.612,5.0,15.0,3.07,True,beta


In [44]:
all_predictor_df['distribution']

0              mu_a
1      a[1988-1989]
2      a[1989-1990]
3      a[1990-1991]
4      a[1991-1992]
           ...     
859       sigma_b18
860       sigma_b19
861       sigma_b20
862       sigma_b21
863       sigma_b22
Name: distribution, Length: 864, dtype: object

In [41]:
beta_hashmap = {}
for i,col in enumerate(numeric_col.name):
    beta = "b"+str(i)
    beta_hashmap[beta] = col
    # something = np.where(all_predictor_df['distribution'].str.contains(beta), name, None)
beta_hashmap

{'b0': 'Age',
 'b1': 'G',
 'b2': 'MP',
 'b3': 'PER',
 'b4': 'TS%',
 'b5': '3PAr',
 'b6': 'FTr',
 'b7': 'ORB%',
 'b8': 'DRB%',
 'b9': 'TRB%',
 'b10': 'AST%',
 'b11': 'STL%',
 'b12': 'BLK%',
 'b13': 'TOV%',
 'b14': 'USG%',
 'b15': 'OWS',
 'b16': 'DWS',
 'b17': 'WS',
 'b18': 'WS/48',
 'b19': 'OBPM',
 'b20': 'DBPM',
 'b21': 'BPM',
 'b22': 'VORP'}

In [51]:
def get_predictors(arr, hashmap):
    new_map = []
    for i in arr:
        predictor_name = ''
        if len(i.split('[')) == 2:
            beta_type = i.split('[')[0]
            if beta_type == 'a':
                predictor_name = 'Intercept'
            else:
                predictor_name = hashmap[beta_type]
        elif 'sigma' in i:
            beta_type = i.split('_')[1]
            if beta_type == 'a':
                # predictor_name = 'Sigma Intercept'
                predictor_name = 'Intercept'
            else:
                # predictor_name = 'Sigma ' + hashmap[beta_type] 
                predictor_name = hashmap[beta_type]
        elif 'mu' in i:
            beta_type = i.split('_')[1]
            if beta_type == 'a':
                # predictor_name = 'Mu Intercept'
                predictor_name = 'Intercept'
            else:
                # predictor_name = 'Mu ' + hashmap[beta_type]
                predictor_name = hashmap[beta_type]
        new_map.append(predictor_name)
    return new_map
all_predictor_df['predictor'] = get_predictors(all_predictor_df['distribution'],beta_hashmap)

In [52]:
all_predictor_df.groupby(['predictor', 'typeDist']).sum()['significant']

predictor  typeDist 
3PAr       beta          3
AST%       beta          6
Age        beta         12
BLK%       beta          7
BPM        beta          4
DBPM       beta         11
DRB%       beta          6
DWS        beta          3
FTr        beta         11
G          beta          6
Intercept  intercept     7
MP         beta          2
OBPM       beta          5
ORB%       beta          4
OWS        beta          5
PER        beta          3
STL%       beta          4
TOV%       beta          4
TRB%       beta          2
TS%        beta          4
USG%       beta          5
VORP       beta          3
WS         beta          6
WS/48      beta          5
Name: significant, dtype: int64

In [53]:
all_predictor_df[all_predictor_df['significant'] == False]

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist,predictor
0,mu_a,0.404,0.459,-0.441,0.862,0.220,0.167,5.0,15.0,3.13,False,intercept,Intercept
2,a[1989-1990],-0.078,0.537,-0.582,0.824,0.257,0.196,5.0,12.0,3.38,False,intercept,Intercept
3,a[1990-1991],-0.256,0.584,-0.687,0.762,0.280,0.213,5.0,13.0,2.28,False,intercept,Intercept
6,a[1993-1994],0.032,0.150,-0.218,0.193,0.071,0.054,6.0,21.0,2.59,False,intercept,Intercept
7,a[1994-1995],-0.353,0.433,-0.846,0.284,0.207,0.158,5.0,25.0,2.66,False,intercept,Intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,b22[2017-2018],0.078,0.651,-0.894,1.084,0.311,0.236,5.0,13.0,3.86,False,beta,VORP
836,b22[2018-2019],-0.097,0.494,-0.977,0.517,0.235,0.179,5.0,12.0,3.13,False,beta,VORP
837,b22[2019-2020],-0.130,0.404,-0.771,0.289,0.191,0.147,5.0,16.0,3.09,False,beta,VORP
838,b22[2020-2021],0.051,0.565,-0.837,0.851,0.269,0.205,5.0,17.0,3.05,False,beta,VORP


In [57]:
np.unique(all_predictor_df[all_predictor_df['significant'] == False]['predictor'], return_counts=True)

(array(['3PAr', 'AST%', 'Age', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr',
        'G', 'Intercept', 'MP', 'OBPM', 'ORB%', 'OWS', 'PER', 'STL%',
        'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS', 'WS/48'], dtype=object),
 array([33, 30, 24, 29, 32, 25, 30, 33, 25, 30, 29, 34, 31, 32, 31, 33, 32,
        32, 34, 32, 31, 33, 30, 31]))

In [54]:
all_predictor_df[all_predictor_df['significant'] == True]

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist,predictor
1,a[1988-1989],0.612,0.327,0.075,0.931,0.156,0.119,5.0,15.0,3.42,True,intercept,Intercept
4,a[1991-1992],0.681,0.184,0.382,0.870,0.086,0.065,5.0,14.0,3.07,True,intercept,Intercept
5,a[1992-1993],0.422,0.228,0.105,0.673,0.109,0.083,5.0,12.0,3.14,True,intercept,Intercept
9,a[1996-1997],-0.703,0.240,-0.962,-0.300,0.115,0.087,5.0,34.0,2.72,True,intercept,Intercept
19,a[2006-2007],0.453,0.237,0.068,0.708,0.112,0.085,5.0,18.0,2.99,True,intercept,Intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,sigma_b18,3.802,2.078,1.509,8.009,0.971,0.736,5.0,15.0,3.14,True,beta,WS/48
860,sigma_b19,3.814,1.535,2.172,5.971,0.726,0.554,5.0,15.0,2.62,True,beta,OBPM
861,sigma_b20,3.577,1.680,1.711,5.789,0.800,0.608,5.0,14.0,2.96,True,beta,DBPM
862,sigma_b21,4.047,1.738,2.498,8.378,0.808,0.612,5.0,15.0,3.07,True,beta,BPM


In [58]:
np.unique(all_predictor_df[all_predictor_df['significant'] == True]['predictor'], return_counts=True)

(array(['3PAr', 'AST%', 'Age', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr',
        'G', 'Intercept', 'MP', 'OBPM', 'ORB%', 'OWS', 'PER', 'STL%',
        'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS', 'WS/48'], dtype=object),
 array([ 3,  6, 12,  7,  4, 11,  6,  3, 11,  6,  7,  2,  5,  4,  5,  3,  4,
         4,  2,  4,  5,  3,  6,  5]))

In [61]:
all_predictor_df[(all_predictor_df['significant'] == True) & (all_predictor_df['predictor'] == 'Age')]

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist,predictor
38,b0[1990-1991],-0.589,0.214,-0.849,-0.211,0.098,0.075,5.0,12.0,2.97,True,beta,Age
42,b0[1994-1995],-0.508,0.205,-0.789,-0.263,0.093,0.070,6.0,14.0,2.27,True,beta,Age
45,b0[1997-1998],-0.372,0.271,-0.758,-0.006,0.125,0.094,5.0,12.0,2.81,True,beta,Age
46,b0[1998-1999],-0.332,0.312,-0.872,-0.028,0.144,0.109,5.0,13.0,2.97,True,beta,Age
48,b0[2000-2001],-0.373,0.239,-0.777,-0.129,0.113,0.086,5.0,12.0,3.66,True,beta,Age
52,b0[2004-2005],-0.541,0.383,-0.974,-0.052,0.173,0.137,5.0,12.0,3.83,True,beta,Age
55,b0[2007-2008],-0.634,0.230,-0.896,-0.271,0.109,0.083,5.0,12.0,3.60,True,beta,Age
59,b0[2011-2012],-0.559,0.227,-0.848,-0.248,0.099,0.075,6.0,18.0,2.22,True,beta,Age
61,b0[2013-2014],-0.721,0.340,-1.190,-0.216,0.161,0.122,5.0,12.0,3.36,True,beta,Age
67,b0[2019-2020],-0.486,0.403,-0.949,-0.045,0.163,0.122,5.0,12.0,2.71,True,beta,Age


In [62]:
all_predictor_df[(all_predictor_df['significant'] == False) & (all_predictor_df['predictor'] == 'Age')]

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist,predictor
35,mu_b0,0.159,0.500,-0.654,0.678,0.239,0.182,5.0,17.0,3.01,False,beta,Age
36,b0[1988-1989],-0.351,0.419,-0.978,0.614,0.190,0.144,5.0,12.0,3.51,False,beta,Age
37,b0[1989-1990],0.153,0.459,-0.620,0.863,0.201,0.153,5.0,15.0,2.21,False,beta,Age
39,b0[1991-1992],-0.282,0.538,-1.024,0.548,0.253,0.192,5.0,13.0,3.80,False,beta,Age
40,b0[1992-1993],-0.078,0.560,-0.717,0.598,0.265,0.201,5.0,12.0,3.91,False,beta,Age
41,b0[1993-1994],0.042,0.519,-0.815,0.963,0.237,0.179,5.0,12.0,3.27,False,beta,Age
43,b0[1995-1996],-0.588,0.462,-1.048,0.065,0.214,0.162,5.0,13.0,2.76,False,beta,Age
44,b0[1996-1997],-0.230,0.277,-0.638,0.155,0.122,0.095,6.0,14.0,2.06,False,beta,Age
47,b0[1999-2000],-0.168,0.461,-0.854,0.263,0.201,0.151,6.0,12.0,2.15,False,beta,Age
49,b0[2001-2002],-0.094,0.321,-0.522,0.248,0.148,0.112,5.0,12.0,2.74,False,beta,Age
